In [17]:
import sys
sys.path.append('../../src')

import pandas as pd
import numpy as np
from tqdm import tqdm
import glob

from run import run
from utils import save_all_columns, save_columns, feather_path

In [18]:
in_columns=["anime_id", "user", "user_count_encoding", "text_len_*", 'episodes', 'members',
       'popularity', 'ranked', 'start_month', 'start_day', 'start_year',
       'end_month', 'end_day', 'end_year', 'len_year', 'len_month', 'len_day',
       'episode_per_month', 'episode_per_day',  'Action',
       'Adventure', 'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi',
       'Fantasy', 'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei',
       'Kids', 'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music',
       'Mystery', 'None', 'Parody', 'Police', 'Psychological', 'Romance',
       'Samurai', 'School', 'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai',
       'Shounen', 'Shounen Ai', 'Slice of Life', 'Space', 'Sports',
       'Super Power', 'Supernatural', 'Thriller', 'Vampire', 'Yaoi', 'Yuri']

In [23]:
test_prefix = "seen"

In [24]:
in_columns_fixed = []
for col in in_columns:
    if "*" in col:
        for filename in sorted(glob.glob(feather_path(col, "train"))):
            in_columns_fixed.append(filename.split("/")[-1].split(".")[0])
    else:
        in_columns_fixed.append(col)

in_columns = in_columns_fixed

In [25]:
test_columns = []
for col in in_columns:
    test_columns.append(test_prefix + "_" + col)

In [26]:
train_df = pd.DataFrame()
pbar = tqdm(in_columns, desc="load train columns")
for col in pbar:
    pbar.set_postfix_str("load {}".format(col))
    df = pd.read_feather(feather_path(col, "train"))
    train_df[col] = df[col].values

load train columns: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 66/66 [00:00<00:00, 488.72it/s, load Yuri]


In [21]:
train_df

,anime_id,user,user_count_encoding,text_len_max,text_len_mean,text_len_min,text_len_std,episodes,members,popularity,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,6574,cd931c240b,10,6202,4518.700000,2212,1290.747677,12.0,57231,1724,...,0,1,0,0,0,0,0,0,0,0
1,12531,bf666ac921,9,7658,5598.888889,3289,1322.178264,12.0,235523,437,...,0,0,0,0,0,0,0,0,0,0
2,23079,24a73e9958,6,5374,3651.166667,2086,1137.507700,13.0,130085,869,...,0,1,0,0,0,1,0,0,0,0
3,8769,d45057b3ec,6,3389,2731.000000,2032,587.834330,12.0,487655,139,...,0,1,0,0,0,0,0,0,0,0
4,12367,9b33e2839e,8,3133,2350.125000,1168,607.952169,12.0,35044,2347,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105344,9756,22815a50b1,1,4990,4990.000000,4990,NaN,12.0,781975,58,...,0,0,0,0,0,0,1,0,0,0
105345,228,52c79b3a9c,1,1438,1438.000000,1438,NaN,26.0,250512,393,...,0,0,0,0,0,1,0,0,0,0
105346,6164,97eab6eb54,1,2356,2356.000000,2356,NaN,11.0,66420,1561,...,0,1,0,0,0,0,0,0,0,0
105347,32,ca1066ee61,4,5181,2745.000000,1901,1624.174252,1.0,402617,208,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_result_df = train.copy()
testseen_result_df = test_seen.copy()

train_mean = train.groupby('user')['score'].mean()
testseen_result_df['user_target_encoding'] = test_seen['user'].map(train_mean)

tmp = np.zeros(train.shape[0])
for (train_idx, val_idx) in kf.split(train):
    target = train.iloc[train_idx]
    target_mean = target.groupby('user')['score'].mean()
    tmp[val_idx] = train['user'].iloc[val_idx].map(target_mean)

train_result_df['user_target_encoding'] = tmp